In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import mat4py # .mat file conversion

from scipy.signal import welch, find_peaks, cwt # Signal Processing

import stumpy # Matrix Profile
from stumpy.floss import _cac # One sided cac

import ot # Optimal Transport

from vmdpy import VMD # Variational Mode Decomposition 

from ruptures.metrics import hausdorff # Quantifying Break Point Discrepancies

# Utils
import time
import os
loc = os.getcwd() 

# Matplotlib Params
import matplotlib
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 16}

matplotlib.rc('font', **font)

# os.chdir('/Users/wang_to/Documents/University/Anomaly_detection/anomalies/bookshelf/dataset')

from matplotlib import animation
from IPython.display import HTML

import scripts

os.chdir('/Users/wang_to/Documents/University/Anomaly_detection/anomalies/bookshelf/dataset')


In [6]:
import importlib
importlib.reload(scripts)

<module 'scripts' from '/Users/wang_to/Documents/University/Anomaly_detection/anomalies/bookshelf/code/streaming/scripts.py'>

# Current Notebook/Pipeline Structure
1.1 - Zero out data from data_dict 

1.2 - Replace with normal data.
Branches 
1. No vmd, only welch, sensor 1. use vmd = 0 in computing building_features and building_features_seq, then animate.
2. Yes vmd[0], sensor 1. Use vmd = num_modes to compute building_features and building_features_seq. Animate.
3. Yes vmd, sensor 1, cac all together
4. Yes vmd, all sensors, cac all sensors together (still to be done)



# Generic Preprocessing Steps (1.1)

5.605706912999988

In [21]:
data_dict['Sensor1'].iloc[0,:-8]

Xt_0       0.000000
Xt_1       0.000000
Xt_2       0.000000
Xt_3       0.000000
Xt_4       0.000000
             ...   
Xt_8183   -0.005531
Xt_8184   -0.019759
Xt_8185   -0.021375
Xt_8186   -0.000613
Xt_8187   -0.002553
Name: 0, Length: 8188, dtype: float64

In [10]:
data_dict = np.load('data_dict.npy', allow_pickle=True).item()
cases = [['L00','D00','V08'], ['L1C','DB0','V08'],  ['L1C','DBB','V08'], ['L13','DBB','V08']]

# for the first 8 sensors, erase some number of points between 100 and 500.
# for other experiments, see what happens if zeroing out is replaced by random normals
num_sensors = 8
bounds = (100, 500)
for i in range(num_sensors):
    sensor_str = f'Sensor{i+1}'
    scripts.temporal_shift(data_dict[sensor_str], bounds, 270)

df_list = scripts.exp_list_process(data_dict, cases, 10)

KeyboardInterrupt: 

In [9]:
df_list

{'Sensor1': Empty DataFrame
 Columns: [(Xt_0,), (Xt_1,), (Xt_2,), (Xt_3,), (Xt_4,), (Xt_5,), (Xt_6,), (Xt_7,), (Xt_8,), (Xt_9,), (Xt_10,), (Xt_11,), (Xt_12,), (Xt_13,), (Xt_14,), (Xt_15,), (Xt_16,), (Xt_17,), (Xt_18,), (Xt_19,), (Xt_20,), (Xt_21,), (Xt_22,), (Xt_23,), (Xt_24,), (Xt_25,), (Xt_26,), (Xt_27,), (Xt_28,), (Xt_29,), (Xt_30,), (Xt_31,), (Xt_32,), (Xt_33,), (Xt_34,), (Xt_35,), (Xt_36,), (Xt_37,), (Xt_38,), (Xt_39,), (Xt_40,), (Xt_41,), (Xt_42,), (Xt_43,), (Xt_44,), (Xt_45,), (Xt_46,), (Xt_47,), (Xt_48,), (Xt_49,), (Xt_50,), (Xt_51,), (Xt_52,), (Xt_53,), (Xt_54,), (Xt_55,), (Xt_56,), (Xt_57,), (Xt_58,), (Xt_59,), (Xt_60,), (Xt_61,), (Xt_62,), (Xt_63,), (Xt_64,), (Xt_65,), (Xt_66,), (Xt_67,), (Xt_68,), (Xt_69,), (Xt_70,), (Xt_71,), (Xt_72,), (Xt_73,), (Xt_74,), (Xt_75,), (Xt_76,), (Xt_77,), (Xt_78,), (Xt_79,), (Xt_80,), (Xt_81,), (Xt_82,), (Xt_83,), (Xt_84,), (Xt_85,), (Xt_86,), (Xt_87,), (Xt_88,), (Xt_89,), (Xt_90,), (Xt_91,), (Xt_92,), (Xt_93,), (Xt_94,), (Xt_95,), (Xt_96,), (

In [111]:
# parameters 

alpha = 2000      # Moderate bandwidth constraint  
tau = 0.          # Noise-tolerance (no strict fidelity enforcement)  
num_modes = 5     # Num modes  
DC = 0            # No DC part imposed  
init = 0          # Initialize omegas uniformly  
tol = 1e-7        # Optimization Convergence Tolerance
fs = 1600
# Parameter for p-welch transform
nperseg = df_list['Sensor1'].shape[1]//8

num_experiments = 25 

# Total length of signal recording
signal_length = 8192 # len(X_cols_time)


# Computing FLOSS algorithm for non-VMD (only welch) (1.1)

In [100]:
building_features = scripts.welch_vmd(df_list['Sensor1'])

In [109]:
building_features_seq = scripts.data_sequencing(building_features, vmd=0)

In [ ]:
regime_change_idxs = [n_exp * 513 for n_exp in [10,15,20]]
start_list_size = 5*513


# Computing CAC FLOSS algorithm for VMD and VMD CAC (1.1), Sensor 1

In [74]:
scripts.welch_vmd(df_list['Sensor1'])

TypeError: 'numpy.ndarray' object is not callable

In [67]:
building_features = scripts.welch_vmd(df_list, 8192, nperseg, fs, 5, alpha, tau, DC, init, tol)
# use these building features for all other cac experiments

Processing Sensor 1
	Processed VMD for Experiment Num (i.e. Data Row) 0
	Processed VMD for Experiment Num (i.e. Data Row) 5
	Processed VMD for Experiment Num (i.e. Data Row) 10
	Processed VMD for Experiment Num (i.e. Data Row) 15
	Processed VMD for Experiment Num (i.e. Data Row) 20
Processing Sensor 2
	Processed VMD for Experiment Num (i.e. Data Row) 0
	Processed VMD for Experiment Num (i.e. Data Row) 5
	Processed VMD for Experiment Num (i.e. Data Row) 10
	Processed VMD for Experiment Num (i.e. Data Row) 15
	Processed VMD for Experiment Num (i.e. Data Row) 20
Processing Sensor 3
	Processed VMD for Experiment Num (i.e. Data Row) 0
	Processed VMD for Experiment Num (i.e. Data Row) 5
	Processed VMD for Experiment Num (i.e. Data Row) 10
	Processed VMD for Experiment Num (i.e. Data Row) 15
	Processed VMD for Experiment Num (i.e. Data Row) 20
Processing Sensor 4
	Processed VMD for Experiment Num (i.e. Data Row) 0
	Processed VMD for Experiment Num (i.e. Data Row) 5
	Processed VMD for Experime